<a href="https://colab.research.google.com/github/CSID-DGU/2024-2-DSCD-Fiveguys-6/blob/main/%ED%81%B4%EB%9F%AC%EC%8A%A4%ED%84%B0%EB%A7%81_%EC%88%9C%EC%84%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sklearn.cluster import KMeans
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.cluster import DBSCAN
from sklearn.metrics.pairwise import haversine_distances
import numpy as np
import pandas as pd

In [ ]:
df_morning = pd.read_csv('/content/drive/MyDrive/DSCD/출근_정류장_좌표_수요.csv', encoding='CP949')
df_morning = df_morning[['ARS_ID', '출근시간_수요', '정류소명', 'X좌표', 'Y좌표']]
df_morning = df_morning.rename(columns = {'X좌표': '경도', 'Y좌표': '위도'})
df_morning = df_morning.drop(index=27)

df_morning

In [ ]:
df_evening = pd.read_csv('/content/drive/MyDrive/DSCD/퇴근_정류장_좌표_수요.csv', encoding='CP949')
df_evening = df_evening[['ARS_ID', '퇴근시간_수요', '정류소명', 'X좌표', 'Y좌표']]
df_evening = df_evening.rename(columns = {'X좌표': '경도', 'Y좌표': '위도'})

df_evening

In [ ]:
# 클러스터링에 사용할 데이터 (위도와 경도)
coordinates_출근 = df_morning[['위도', '경도']]

# K-Means 클러스터링 수행
num_clusters_출근 = 4  # 원하는 클러스터 개수
kmeans_출근 = KMeans(n_clusters=num_clusters_출근, random_state=0)
df_morning['클러스터'] = kmeans_출근.fit_predict(coordinates_출근)

# 결과 출력
df_morning

In [ ]:
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

# KMeans 실행 및 Inertia 계산
inertia_values = []  # Inertia 값 저장
k_values = range(1, 11)  # k=1에서 k=10까지 실험

for k in k_values:
    kmeans = KMeans(n_clusters=k, random_state=0)
    kmeans.fit(coordinates_출근)
    inertia_values.append(kmeans.inertia_)  # Inertia 값 저장

# 결과 시각화 (엘보우 기법)
plt.figure(figsize=(8, 5))
plt.plot(k_values, inertia_values, marker='o')
plt.title('Elbow Method (Inertia vs. Number of Clusters)')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('Inertia')
plt.grid()
plt.show()


In [ ]:
import folium

# Folium 지도 생성
map_center = [df_morning['위도'].mean(), df_morning['경도'].mean()]
m = folium.Map(location=map_center, zoom_start=13)

# 클러스터링 결과를 지도에 추가
colors = ['red', 'blue', 'green', 'purple', 'orange', 'darkred', 'lightblue', 'darkgreen', 'pink', 'cadetblue']
colors_출근 = colors[:num_clusters_출근]
for _, row in df_morning.iterrows():
    cluster = row['클러스터']
    color = colors_출근[cluster % len(colors)]  # 클러스터별 색상 할당
    folium.CircleMarker(
        location=[row['위도'], row['경도']],
        radius=8,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.7,
        popup=f"Station: {row['ARS_ID']}, Cluster: {cluster}"
    ).add_to(m)

# 결과 지도 저장
m.save('clustering_result_map_출근_홍대제외.html')

In [ ]:
# 클러스터링에 사용할 데이터 (위도와 경도)
coordinates_퇴근 = df_evening[['위도', '경도']]

# K-Means 클러스터링 수행
num_clusters_퇴근 = 5  # 원하는 클러스터 개수
kmeans_퇴근 = KMeans(n_clusters=num_clusters_퇴근, random_state=0)
df_evening['클러스터'] = kmeans_퇴근.fit_predict(coordinates_퇴근)

# 결과 출력
df_evening

In [ ]:
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

# KMeans 실행 및 Inertia 계산
inertia_values = []  # Inertia 값 저장
k_values = range(1, 11)  # k=1에서 k=10까지 실험

for k in k_values:
    kmeans = KMeans(n_clusters=k, random_state=0)
    kmeans.fit(coordinates_퇴근)
    inertia_values.append(kmeans.inertia_)  # Inertia 값 저장

# 결과 시각화 (엘보우 기법)
plt.figure(figsize=(8, 5))
plt.plot(k_values, inertia_values, marker='o')
plt.title('Elbow Method (Inertia vs. Number of Clusters)')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('Inertia')
plt.grid()
plt.show()


In [ ]:
import folium

# Folium 지도 생성
map_center = [df_evening['위도'].mean(), df_evening['경도'].mean()]
m = folium.Map(location=map_center, zoom_start=13)

# 클러스터링 결과를 지도에 추가
colors = ['red', 'blue', 'green', 'purple', 'orange', 'darkred', 'lightblue', 'darkgreen', 'pink', 'cadetblue']
colors_출근 = colors[:num_clusters_퇴근]
for _, row in df_evening.iterrows():
    cluster = row['클러스터']
    color = colors[cluster % len(colors)]  # 클러스터별 색상 할당
    folium.CircleMarker(
        location=[row['위도'], row['경도']],
        radius=8,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.7,
        popup=f"Station: {row['ARS_ID']}, Cluster: {cluster}"
    ).add_to(m)

# 결과 지도 저장
m.save('clustering_result_map_퇴근.html')

In [ ]:
def find_closest_point_between_clusters(cluster1, cluster2):
    """
    두 클러스터 간에서 가장 가까운 두 점을 반환.
    cluster1에서 선택된 점은 반드시 cluster1에 속하며,
    cluster2에서 선택된 점은 반드시 cluster2에 속합니다.
    """
    cluster1_coords = cluster1[['위도', '경도']].values
    cluster2_coords = cluster2[['위도', '경도']].values
    distances = cdist(cluster1_coords, cluster2_coords, metric='euclidean')
    i, j = np.unravel_index(np.argmin(distances), distances.shape)
    return cluster1.iloc[i], cluster2.iloc[j]


def find_farthest_point(cluster_df, reference_point):
    """클러스터 내에서 reference_point와 가장 먼 점 찾기"""
    # 수치형 변환 (명시적으로 float64로 변환)
    reference_point = np.array(reference_point, dtype=np.float64).reshape(1, -1)
    distances = cdist(reference_point, cluster_df[['위도', '경도']].values, metric='euclidean')
    farthest_idx = np.argmax(distances)
    return cluster_df.iloc[farthest_idx]

def find_closest_point(cluster_df, reference_point):
    """클러스터 내에서 reference_point와 가장 가까운 점 찾기"""
    # reference_point가 객체가 아닌 수치형인지 확인 후 변환
    reference_point = np.array(reference_point, dtype=np.float64).reshape(1, -1)
    distances = cdist(reference_point, cluster_df[['위도', '경도']].values, metric='euclidean')
    closest_idx = np.argmin(distances)
    return cluster_df.iloc[closest_idx]

In [ ]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist
from itertools import permutations

# 출근 클러스터 순서 결정
# 3. 클러스터 간 최적 순서 결정 (TSP 방식)
cluster_centroids_출근 = pd.DataFrame(kmeans_출근.cluster_centers_, columns=['위도', '경도'])
cluster_centroids_출근['클러스터'] = range(num_clusters_출근)

def calculate_tsp_route(locations):
    """단순 TSP로 최단 경로 계산"""
    coords = locations[['위도', '경도']].values
    min_distance = float('inf')
    best_route = None
    for perm in permutations(range(len(coords))):
        dist = 0
        for i in range(len(perm) - 1):
            dist += np.linalg.norm(coords[perm[i]] - coords[perm[i + 1]])
        if dist < min_distance:
            min_distance = dist
            best_route = perm
    return best_route, min_distance

# 클러스터 중심들로 최단 경로 계산
route_출근, route_distance_출근 = calculate_tsp_route(cluster_centroids_출근)

In [ ]:
# 클러스터 경계 최적화 (출근)
final_route = []
for i in range(len(route_출근)):
    current_cluster = df_morning[df_morning['클러스터'] == route_출근[i]]

    if i == 0:  # 첫 클러스터
        # 첫 클러스터의 끝점: 다음 클러스터와 가장 가까운 점
        next_cluster = df_morning[df_morning['클러스터'] == route_출근[i + 1]]
        end_point, _ = find_closest_point_between_clusters(current_cluster, next_cluster)

        # 첫 클러스터의 시작점: 끝점과 가장 먼 점
        start_point = find_farthest_point(current_cluster, end_point[['위도', '경도']].values)

    elif i == len(route_출근) - 1:  # 마지막 클러스터
        # 마지막 클러스터의 시작점: 이전 클러스터 끝점과 가장 가까운 점
        prev_coords = np.array(final_route[-1]['끝점'][['위도', '경도']].values, dtype=np.float64).reshape(1, -1)
        start_point = find_closest_point(current_cluster, prev_coords)

        # 마지막 클러스터의 끝점: 시작점과 가장 먼 점
        end_point = find_farthest_point(current_cluster, start_point[['위도', '경도']].values)

    else:  # 중간 클러스터
        # 시작점: 이전 클러스터 끝점과 가장 가까운 점
        prev_coords = np.array(final_route[-1]['끝점'][['위도', '경도']].values, dtype=np.float64).reshape(1, -1)
        start_point = find_closest_point(current_cluster, prev_coords)

        # 끝점: 다음 클러스터와 가장 가까운 점
        next_cluster = df_morning[df_morning['클러스터'] == route_출근[i + 1]]
        end_point, _ = find_closest_point_between_clusters(current_cluster, next_cluster)

    final_route.append({'클러스터': route_출근[i], '시작점': start_point, '끝점': end_point})

# 결과 출력
print("출근시간 최적의 클러스터 간 순서:")
print(" -> ".join([f"클러스터 {cluster_centroids_출근.iloc[i]['클러스터']}" for i in route_출근]))
print("\n클러스터 경계 정류장:")
for item in final_route:
    print(f"클러스터 {item['클러스터']}: 시작점 - {item['시작점']['ARS_ID']}, 끝점 - {item['끝점']['ARS_ID']}")

In [ ]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist
from itertools import permutations


# 3. 클러스터 간 최적 순서 결정 (TSP 방식)
cluster_centroids_퇴근 = pd.DataFrame(kmeans_퇴근.cluster_centers_, columns=['위도', '경도'])
cluster_centroids_퇴근['클러스터'] = range(num_clusters_퇴근)

def calculate_tsp_route(cluster_centroids):
    """단순 TSP로 최단 경로 계산"""
    coords = cluster_centroids[['위도', '경도']].values
    min_distance = float('inf')
    best_route = None
    for perm in permutations(range(len(coords))):
        dist = 0
        for i in range(len(perm) - 1):
            dist += np.linalg.norm(coords[perm[i]] - coords[perm[i + 1]])
        if dist < min_distance:
            min_distance = dist
            best_route = perm
    return best_route, min_distance

# 클러스터 중심들로 최단 경로 계산
route_퇴근, route_distance_퇴근 = calculate_tsp_route(cluster_centroids_퇴근)

In [ ]:
# 클러스터 경계 최적화 (퇴근)
final_route = []
for i in range(len(route_퇴근)):
    current_cluster = df_evening[df_evening['클러스터'] == route_퇴근[i]]

    if i == 0:  # 첫 클러스터
        # 첫 클러스터의 끝점: 다음 클러스터와 가장 가까운 점
        next_cluster = df_evening[df_evening['클러스터'] == route_퇴근[i + 1]]
        end_point, _ = find_closest_point_between_clusters(current_cluster, next_cluster)

        # 첫 클러스터의 시작점: 끝점과 가장 먼 점
        start_point = find_farthest_point(current_cluster, end_point[['위도', '경도']].values)

    elif i == len(route_퇴근) - 1:  # 마지막 클러스터
        # 마지막 클러스터의 시작점: 이전 클러스터 끝점과 가장 가까운 점
        prev_coords = np.array(final_route[-1]['끝점'][['위도', '경도']].values, dtype=np.float64).reshape(1, -1)
        start_point = find_closest_point(current_cluster, prev_coords)

        # 마지막 클러스터의 끝점: 시작점과 가장 먼 점
        end_point = find_farthest_point(current_cluster, start_point[['위도', '경도']].values)

    else:  # 중간 클러스터
        # 시작점: 이전 클러스터 끝점과 가장 가까운 점
        prev_coords = np.array(final_route[-1]['끝점'][['위도', '경도']].values, dtype=np.float64).reshape(1, -1)
        start_point = find_closest_point(current_cluster, prev_coords)

        # 끝점: 다음 클러스터와 가장 가까운 점
        next_cluster = df_evening[df_evening['클러스터'] == route_퇴근[i + 1]]
        end_point, _ = find_closest_point_between_clusters(current_cluster, next_cluster)

    final_route.append({'클러스터': route_퇴근[i], '시작점': start_point, '끝점': end_point})

# 결과 출력
print("퇴근시간 최적의 클러스터 간 순서:")
print(" -> ".join([f"클러스터 {cluster_centroids_퇴근.iloc[i]['클러스터']}" for i in route_퇴근]))
print("\n클러스터 경계 정류장:")
for item in final_route:
    print(f"클러스터 {item['클러스터']}: 시작점 - {item['시작점']['ARS_ID']}, 끝점 - {item['끝점']['ARS_ID']}")